<a href="https://colab.research.google.com/github/marcusvpimentel/Mestrado/blob/main/BaggingClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris
dataset = load_iris()
X, y = dataset.data, dataset.target

In [4]:
import random
import numpy as np
def sorteiaCaracteristica(X, y):
  caracteristica = random.randint(a=0, b=X.shape[1]-1)
  valor = random.uniform(a=np.min(X[:,caracteristica]), b=np.max(X[:,caracteristica]))
  return caracteristica, valor
sorteiaCaracteristica(X, y)

(1, 3.32726948959164)

In [5]:
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
from sklearn.model_selection import cross_validate
from sklearn.ensemble import BaggingClassifier

def maisFrequente(y):
  return Counter(y.flat).most_common(1)[0][0]

class Arvore(BaseEstimator, ClassifierMixin):
  def fit(self, X, y):
    self.caracteristica, self.valor = sorteiaCaracteristica(X, y)
    maiores = X[:,self.caracteristica] > self.valor
    if sum(maiores)>0 and sum(~maiores)>0:
      self.maiores = Arvore()
      self.maiores.fit(X[maiores,:], y[maiores])
      self.menores = Arvore()
      self.menores.fit(X[~maiores,:], y[~maiores])
    else:
      self.resposta = maisFrequente( y )
    return self
  def predict(self, X):
    y = np.empty((X.shape[0]))
    if hasattr(self, 'resposta'):
      y[:] = self.resposta
    else:
      maiores = X[:, self.caracteristica] > self.valor
      y[maiores] = self.maiores.predict(X[maiores,:])
      y[~maiores] = self.menores.predict(X[~maiores,:])
    return y

    def fit(self, X, y):
        self.caracteristica, self.valor = self.sorteiaCaracteristica(X, y)
        maiores = X[:,self.caracteristica] > self.valor
        if np.sum(maiores) > 0 and np.sum(~maiores) > 0:
            self.maiores = Arvore()
            self.maiores.fit(X[maiores,:], y[maiores])
            self.menores = Arvore()
            self.menores.fit(X[~maiores,:], y[~maiores])
        else:
            self.resposta = self.maisFrequente(y)
        return self

    def predict(self, X):
        y = np.empty((X.shape[0]))
        if self.resposta is not None:
            y[:] = self.resposta
        else:
            maiores = X[:, self.caracteristica] > self.valor
            y[maiores] = self.maiores.predict(X[maiores,:])
            y[~maiores] = self.menores.predict(X[~maiores,:])
        return y



In [6]:
results = cross_validate(Arvore(), X, y)
results['test_score'], np.mean(results['test_score'])

(array([0.93333333, 0.9       , 0.9       , 0.93333333, 0.96666667]),
 0.9266666666666667)

In [7]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

modelo = BaggingClassifier(DecisionTreeClassifier(splitter='random'),
                           n_estimators=100)
results = cross_validate(modelo, X, y)
results['test_score'], np.mean(results['test_score'])

(array([0.96666667, 0.96666667, 0.93333333, 0.9       , 1.        ]),
 0.9533333333333334)